In [90]:
import dml
from pymongo import MongoClient
import pandas as pd
from scipy.stats.stats import pearsonr  
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import numpy as np

In [106]:
client = MongoClient()
db = client.repo
collection = db.jhs2018_rpm1995.kmeansdata
df = pd.DataFrame(list(collection.find()))
df = df[(df.charge_count != 0)|(df.hubway_count != 0)|(df.open_count != 0)]
df = df.reset_index()
latitude = [0]*len(df)
longitude = [0]*len(df)
for i in range(len(df)):
    latitude[i] = df['coordinates'][i][0]
    longitude[i] = df['coordinates'][i][1]
    

df['latitude'] = latitude
df['longitude']= longitude
#df['reallat'] = reallat
#df['reallong'] = reallong
df = df[(df.charge_count != 0)|(df.hubway_count != 0)|(df.open_count != 0)]

df[['charge_count','hubway_count','open_count','latitude','longitude']] = MinMaxScaler().fit_transform(df[['charge_count','hubway_count','open_count','latitude','longitude']])

df['latitude'] = df.latitude + 1.0
df['longitude'] = df.longitude + 1.0
df['charge_count'] = df.charge_count
df['hubway_count'] = dft.hubway_count
df['open_count'] = df.open_count
#df = df.reset_index()


In [120]:
y_pred = KMeans(n_clusters = 8, random_state = 0)
y_pred = y_pred.fit_predict(df[['charge_count','hubway_count','open_count','latitude','longitude']])

df['clusters'] = y_pred

budgets = df.groupby('clusters')['budget'].apply(list)
open_count = df.groupby('clusters')['open_count'].apply(list)

hubway_count = df.groupby('clusters')['hubway_count'].apply(list)
charge_count = df.groupby('clusters')['charge_count'].apply(list)

tree_count = df.groupby('clusters')['tree_count'].apply(list)
#print(pearsonr(open_count,budget))

In [121]:
for i in range(len(budgets)):
    print (pearsonr(open_count[i], budgets[i]))

(0.1848541471276422, 0.23534157363172278)
(-0.05924721033325769, 0.8096084792087838)
(0.5333145295245244, 0.004175836372515903)
(-0.14480639315384883, 0.3425846759458493)
(-0.008386355445558694, 0.9704532122936871)
(0.39304674142460766, 0.03492303274763899)
(nan, nan)
(0.3202563076101743, 0.2445392170835299)


C:\Users\Jonathan\Anaconda2\envs\py36\lib\site-packages\scipy\stats\stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
C:\Users\Jonathan\Anaconda2\envs\py36\lib\site-packages\scipy\stats\stats.py:5240: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


In [98]:
def get_means_for_clusters(listvalues):
    means = []
    for listval in listvalues:
        means.append(np.mean(listval))
    return means
budgetmeans = get_means_for_clusters(budgets)
openspacemeans = get_means_for_clusters(open_count)
treecountmeans = get_means_for_clusters(tree_count)

In [99]:
print (pearsonr(budgetmeans, tree_count))

(0.21806182795867107, 0.5194837916721345)
